In [1]:
import sys
sys.path.append("..")   # this allows us to import from sibling directory

from code_to_import.dataset_preprocessing import ProductionModeDataset

import torch
from sklearn.inspection import permutation_importance
import numpy as np
import uproot
import torch.nn as nn
from matplotlib import pyplot as plt

from code_to_import.Classifier_module import Classifier
from code_to_import.Classifier_module import Three_Layer_Classifier
from torch.autograd import Variable


In [2]:
################# THIS IS WHERE YOU CHOOSE WHAT TO LOAD ################################

correlation_cut = 0.9   # if negative none, current other options are 0.9 and 0.8, but more can be added

cut_version = -1   # -1 means OG, otherwise can be 2 or 3 (or more if added)

event_type = "ee"  # could be ee, emu, mumu

model_name = "threeLayerModel_" + event_type + "_corrCut_" + str(correlation_cut)

if cut_version > 0:
    model_name += "cutV" + str(args.cut_version)

In [3]:
root_path = "/depot-new/cms/top/mcnama20/TopSpinCorr-Run2-Entanglement/CMSSW_10_2_22/src/TopAnalysis/Configuration/analysis/diLeptonic/three_files/Nominal"

file = root_path + "/" + event_type + "_modified_root_1.root"

path_to_model = "../models/three_layers/" + model_name + ".pt"

In [4]:
# data_object = ProductionModeDataset(file)
# x = data_object.get_eval_data()

x = np.load("../data/three_layers/train_val_datasets/validation_dataset_" + model_name + ".npy")


In [5]:
############ remove parts of dataset not used in nn input and get output goals (target) and weights ####################
l = x.shape[1]   # get the initial length of the validation dataset
weight = x[:,l-2]   # get an array of weights for plotting
target = x[:,l-3]   # get targets
target = Variable(torch.from_numpy(target).type(torch.LongTensor))   # reformate target into a tensor

######################### remove excess inputs from x and call new variable y
y = np.transpose(x)   
y = np.delete(y, [l-3, l-2, l-1], 0)
y = np.transpose(y)

# the final validation input is gotten from turning y into a tensor
input = Variable(torch.from_numpy(y).type(torch.Tensor))

In [6]:
############################## Load the model and run eval so we know to evaluate ##################################
try:
    model = Classifier()
    model.load_state_dict(torch.load(path_to_model))
except:
    model = Three_Layer_Classifier(input_size=input.shape[1])
    model.load_state_dict(torch.load(path_to_model))
model.eval()

Three_Layer_Classifier(
  (model): Sequential(
    (0): Linear(in_features=77, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=256, out_features=3, bias=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
  )
)

In [7]:
input.requires_grad=True

In [8]:
# optimizer = torch.optim.Adam(model.parameters(), , betas=(opt.b1, opt.b2)) 
# criterion = nn.MSELoss()   # mean squared error loss
criterion = nn.CrossEntropyLoss()


In [9]:
# model.retain_grad()
input.retain_grad()
output = model(input).requires_grad_(True)
output.retain_grad()

loss = criterion(output, target)
loss.retain_grad()
print(loss)

# external_grad = torch.tensor(1.0)
loss.backward()

tensor(1.0343, grad_fn=<NllLossBackward>)


In [10]:
print(input.grad)

tensor([[-7.8111e-05, -1.7433e-05, -7.1622e-06,  ...,  2.7651e-05,
          1.8326e-05,  6.9812e-05],
        [ 2.0512e-04,  5.0996e-05,  1.2834e-05,  ..., -1.2742e-05,
          1.2689e-05, -2.4365e-04],
        [-3.1547e-05, -1.6176e-05,  2.2719e-05,  ...,  5.7186e-05,
          9.5292e-06,  1.6379e-05],
        ...,
        [-9.8339e-05, -6.5114e-06, -4.8011e-05,  ..., -3.2148e-05,
         -1.9842e-05,  2.1551e-04],
        [-1.9313e-05,  2.7914e-07,  1.3743e-05,  ...,  3.7823e-05,
          1.8058e-05,  8.9856e-06],
        [ 1.6170e-04,  5.6701e-05,  2.2406e-05,  ..., -1.9096e-05,
          1.1472e-06, -2.0618e-04]])


In [11]:
mean_grad = input.grad.detach().numpy().mean(axis=0)
max_grad = np.max(mean_grad)

mean_grad = mean_grad/max_grad
mean_grad

array([ 8.71550739e-02,  1.27246156e-02,  3.76641527e-02,  2.70680171e-02,
       -4.73798066e-03, -3.17482352e-02,  9.23438929e-04,  1.28797561e-01,
       -2.30196323e-02, -3.99974957e-02,  1.10416710e-01, -3.00330333e-02,
        6.42948002e-02, -5.17671816e-02,  4.75413762e-02, -3.45017426e-02,
       -3.89895625e-02,  4.99210227e-03, -1.22146010e-02, -3.28041427e-02,
        6.39286339e-02, -2.70720124e-02,  3.89502570e-02, -5.45243919e-03,
       -5.49065396e-02, -3.07842344e-02, -3.72477807e-02, -3.18398955e-03,
       -3.02875247e-02, -2.23929472e-02,  9.78457648e-03,  3.32857072e-02,
        4.18165103e-02,  1.26474351e-01,  1.68744437e-02,  6.89268708e-02,
        4.64608818e-02,  2.20021196e-02,  1.49293169e-02, -2.04525832e-02,
       -1.28714703e-02,  5.07334173e-02,  4.31751125e-02, -4.56997126e-01,
       -7.12386239e-03, -5.08777201e-02, -6.67413091e-03,  3.14221568e-02,
       -1.71417110e-02, -2.19892003e-02,  4.06832621e-02, -4.79362532e-02,
       -7.93394446e-03,  

In [12]:
std_grad = np.std(input.grad.detach().numpy(), axis=0)/np.sqrt(input.shape[0])
std_grad

array([1.8572448e-06, 6.3682188e-07, 4.7199913e-07, 8.6314827e-07,
       7.0967752e-07, 1.3599542e-06, 6.4463092e-07, 1.8536072e-06,
       4.1937301e-07, 4.1389222e-07, 7.1745927e-07, 2.6862378e-07,
       4.6280098e-07, 5.1251311e-07, 6.5755262e-07, 3.0163315e-07,
       5.0457851e-07, 2.2650727e-07, 4.1211072e-07, 3.3188209e-07,
       6.6374423e-07, 4.6614338e-07, 4.4260804e-07, 4.5300359e-07,
       8.5889462e-07, 4.1842054e-07, 3.5711707e-07, 6.5599966e-07,
       7.2766028e-07, 5.0608088e-07, 3.6143749e-07, 6.7866023e-07,
       5.1321786e-07, 1.4398336e-06, 3.0103075e-07, 2.9499688e-06,
       3.9449128e-07, 7.5510457e-07, 4.3088431e-07, 3.2753090e-07,
       3.0009491e-07, 4.2165149e-07, 4.5764429e-07, 4.9110340e-06,
       3.7800589e-07, 1.2855087e-06, 3.1324976e-07, 3.7301342e-07,
       2.1391463e-07, 2.8006227e-07, 5.0049289e-07, 7.2824105e-07,
       3.6743259e-07, 3.4444125e-07, 4.0666976e-07, 4.6961617e-07,
       3.5396141e-07, 3.0098181e-07, 3.6513845e-07, 4.7363730e

In [13]:
events = uproot.open(file)
key = events.keys()[0]
input_variables = events[key].keys()

In [14]:
for i in mean_grad.argsort()[::-1]:
#     if mean_grad[i] - 2 * r.importances_std[i] > 0:
    print(f"{input_variables[i]:<8}                    "
          f"{mean_grad[i]:.3f}"
          f" +/- {std_grad[i]:.3f}")


nu_pt                       1.000 +/- 0.000
nubar_pt                    0.615 +/- 0.000
lbarnubar_delta_eta                    0.129 +/- 0.000
top_boosted_eta                    0.126 +/- 0.000
bbarnu_delta_eta                    0.110 +/- 0.000
top_pt                      0.102 +/- 0.000
lb_delta_eta                    0.087 +/- 0.000
wminus_eta                    0.078 +/- 0.000
bbar_pt                     0.078 +/- 0.000
tbar_pt                     0.076 +/- 0.000
tbar_boosted_eta                    0.069 +/- 0.000
wplus_phi                    0.066 +/- 0.000
lb_delta_phi                    0.064 +/- 0.000
bnu_delta_phi                    0.064 +/- 0.000
l_pt                        0.062 +/- 0.000
top_phi                     0.051 +/- 0.000
l_phi                       0.050 +/- 0.000
lnu_delta_phi                    0.048 +/- 0.000
ttbar_delta_eta                    0.046 +/- 0.000
tbar_phi                    0.043 +/- 0.000
top_eta                     0.042 +/- 0.000
nunubar_delta_

In [15]:
np.save("./results/derivative_importance" + model_name + ".npy", mean_grad)
np.save("./results/derivative_std" + model_name + ".npy", std_grad)